In [11]:
#!pip install wandb

In [12]:
#!pip install trl transformers

In [13]:
#!pip install torch

In [14]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset, load_dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

wandb.finish()
os.environ['WANDB_DISABLED'] = 'true'

# dataset = load_dataset("imdb", split="train")

# if torch.backends.mps.is_built():
#     device = torch.device("mps")

model_name = 'AnatoliiPotapov/T-lite-instruct-0.1'

In [15]:
texts = [
    {"role": "user", "content": "Какова текущая ситуация с авиаперелетами в Европу?"},
    {"role": "bot", "content": "На данный момент авиаперелеты в Европу выполняются по расписанию, однако из-за погодных условий возможны небольшие задержки."},
    
    {"role": "user", "content": "Какой лучший способ добраться до Парижа из Лондона?"},
    {"role": "bot", "content": "Лучший способ добраться до Парижа из Лондона — это поезд Eurostar, который занимает около 2 часов 20 минут."},
    
    {"role": "user", "content": "Какие документы нужны для поездки в США?"},
    {"role": "bot", "content": "Для поездки в США потребуется действующий паспорт и виза. Если вы путешествуете по программе безвизового въезда, потребуется также одобрение ESTA."},
    
    {"role": "user", "content": "Как выбрать лучший отель для семейного отдыха?"},
    {"role": "bot", "content": "При выборе отеля для семейного отдыха важно учитывать удобства для детей, наличие бассейна, анимации и семейных номеров."},
    
    {"role": "user", "content": "Какие направления для отдыха популярны этой зимой?"},
    {"role": "bot", "content": "Этой зимой популярны направления для отдыха, такие как горнолыжные курорты Швейцарии и Австрии, а также теплые страны, такие как Таиланд и Мальдивы."},
    
    {"role": "user", "content": "Какой лучший способ забронировать тур по Европе?"},
    {"role": "bot", "content": "Лучший способ забронировать тур по Европе — это воспользоваться услугами туристического агентства или использовать специализированные онлайн-платформы для самостоятельного бронирования."},
    
    {"role": "user", "content": "Какие туристические достопримечательности обязательно стоит посетить в Риме?"},
    {"role": "bot", "content": "В Риме обязательно стоит посетить Колизей, Ватикан, Пантеон и фонтан Треви."},
    
    {"role": "user", "content": "Как подготовиться к длительному перелету?"},
    {"role": "bot", "content": "Чтобы подготовиться к длительному перелету, рекомендуется одеваться комфортно, взять с собой развлечения, пить воду и регулярно двигаться во время полета."},
    
    {"role": "user", "content": "Какой сезон лучше всего подходит для посещения Японии?"},
    {"role": "bot", "content": "Лучший сезон для посещения Японии — это весна (апрель-май) из-за цветения сакуры, а также осень (сентябрь-ноябрь), когда можно насладиться яркими осенними пейзажами."},
    
    {"role": "user", "content": "Какие визовые требования существуют для поездки в Австралию?"},
    {"role": "bot", "content": "Для поездки в Австралию необходимо получить туристическую визу, которую можно оформить онлайн через сайт правительства Австралии."},
    
    {"role": "user", "content": "Какие меры безопасности необходимо соблюдать при поездках в экзотические страны?"},
    {"role": "bot", "content": "При поездках в экзотические страны важно следить за качеством воды и пищи, использовать репелленты от насекомых и соблюдать местные рекомендации по здоровью."},
    
    {"role": "user", "content": "Какие пляжи считаются самыми красивыми в мире?"},
    {"role": "bot", "content": "Самыми красивыми пляжами в мире считаются пляжи на Мальдивах, Сейшельских островах, Бора-Бора и острове Мауи в Гавайях."},
    
    {"role": "user", "content": "Какой бюджет нужен для поездки по Южной Америке?"},
    {"role": "bot", "content": "Бюджет для поездки по Южной Америке зависит от стран, которые вы планируете посетить. Средний бюджет для путешествия может составлять от $50 до $150 в день, включая проживание, транспорт и питание."},
    
    {"role": "user", "content": "Какие экскурсии можно посетить в Барселоне?"},
    {"role": "bot", "content": "В Барселоне можно посетить экскурсии по Саграда Фамилия, Парк Гуэль, Готический квартал и Дом Бальо."},
    
    {"role": "user", "content": "Как выбрать направление для медового месяца?"},
    {"role": "bot", "content": "При выборе направления для медового месяца учитывайте ваши предпочтения: спокойные пляжные курорты, такие как Мальдивы и Сейшелы, или культурные и активные поездки, например в Италию или Францию."},
    
    {"role": "user", "content": "Какие меры предосторожности следует принять при аренде автомобиля за границей?"},
    {"role": "bot", "content": "При аренде автомобиля за границей убедитесь, что у вас есть действующие водительские права, международная страховка, и обязательно проверяйте условия аренды на наличие скрытых платежей."},
    
    {"role": "user", "content": "Какие направления считаются самыми доступными для путешествий по Европе?"},
    {"role": "bot", "content": "Самыми доступными направлениями для путешествий по Европе считаются Польша, Венгрия, Португалия и Чехия."},
    
    {"role": "user", "content": "Какие виды активного отдыха популярны на Бали?"},
    {"role": "bot", "content": "На Бали популярны такие виды активного отдыха, как серфинг, йога, дайвинг и пешие прогулки по вулканам."},
    
    {"role": "user", "content": "Как избежать перегруженных туристических мест?"},
    {"role": "bot", "content": "Чтобы избежать перегруженных туристических мест, планируйте посещения на будние дни, выбирайте менее известные достопримечательности и путешествуйте вне сезона."},
    
    {"role": "user", "content": "Какие страны требуют вакцинации перед поездкой?"},
    {"role": "bot", "content": "Перед поездкой в некоторые страны, такие как Бразилия, Таиланд или страны Африки, могут потребоваться вакцинации от жёлтой лихорадки, гепатита или малярии."}
]


In [16]:
# Преобразование в формат id, source, message
formatted_texts = []
id_counter = 1

for i in range(0, len(texts), 2):
    user_message = texts[i]['content']
    bot_message = texts[i + 1]['content']
    
    formatted_texts.append({
        "id": id_counter,
        "source": "user",
        "message": user_message
    })
    
    formatted_texts.append({
        "id": id_counter,
        "source": "bot",
        "message": bot_message
    })
    
    id_counter += 1

In [17]:
df = pd.DataFrame(
    {
        'id': [item['id'] for item in formatted_texts],
        'source': [item['source'] for item in formatted_texts],
        'message': [item['message'] for item in formatted_texts]
    }
)
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [18]:
# model = AutoModelForCausalLM.from_pretrained(model_name)
# sft_config = TrainingArguments(output_dir='test_trainer', use_cpu=True)

# trainer = SFTTrainer(
#     model,
#     train_dataset=hg_dataset,
#     args=sft_config,
#     # args=SFTConfig(),
#     dataset_text_field="message",
#     packing=False,
#     max_seq_length=512
# )

# # model.config.use_cache = False
# trainer.train()

In [19]:
# !pip install trl # PEFT
# !pip install git+https://github.com/huggingface/peft.git # PEFT
# !pip install -U git+https://github.com/huggingface/transformers.git # flash-attention

import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset, load_dataset
from peft import LoraConfig
from trl import ModelConfig, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

model_name = 'AnatoliiPotapov/T-lite-instruct-0.1'

model_config = ModelConfig(
    model_name_or_path=model_name,
    attn_implementation='flash_attention_2'
)

#device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name)
sft_config = TrainingArguments(output_dir='travel_trainer', use_cpu=True)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model,
    train_dataset=hg_dataset,
    args=sft_config,
    dataset_text_field="message",
    packing=False,
    peft_config=peft_config,
    max_seq_length=256
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-5y93hko4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-5y93hko4
  Resolved https://github.com/huggingface/peft.git to commit b180ae46f8cf9663f3bf786b41b2eb41c2512031
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-e640yn3d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-e640yn3d
  Resolved https://github.com/huggingface/transformers.git to commit 66bc4def9505fa7c7fe4aa7a248c34a026bb552b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/user1/environments/hack/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, t

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=15, training_loss=1.5340161641438803, metrics={'train_runtime': 154.0724, 'train_samples_per_second': 0.779, 'train_steps_per_second': 0.097, 'total_flos': 291696381001728.0, 'train_loss': 1.5340161641438803, 'epoch': 3.0})